In [ ]:
import os
from matplotlib import pyplot
import pandas as pd
from random import choices

from autumn.tools import db
from autumn.tools.project import get_project
from autumn.settings import Models, Region
from autumn.settings.folders import OUTPUT_DATA_PATH

In [ ]:
def get_param_dict_from_run_index(params_df, run):
    """
    Pull out the part of the parameters dataframe that corresponds to the run index of interest and 
    create a parameter dictionary from that single parameter set.
    """
    run_df = params_df[params_df.run == run]
    return {
        param_name: param_value for 
        param_name, param_value in 
        zip(run_df.name, run_df.value)
    }

In [ ]:
# A few parameters to play with
model = Models.COVID_19
region = Region.VICTORIA_2020
dirname = "2021-08-25"  # Currently this is an arbitrary local folder name for the calibrated run we want to use
burn_in = 0
n_select = 10

In [ ]:
# Load everything up
project = get_project(model, region)
calib_path = os.path.join(OUTPUT_DATA_PATH, "calibrate", project.model_name, project.region_name, dirname)
mcmc_tables = db.load.load_mcmc_tables(calib_path)
mcmc_params = db.load.load_mcmc_params_tables(calib_path)

# I think this is just about data structures not being ideal
mcmc_df = db.load.append_tables(mcmc_tables)
params_df = db.load.append_tables(mcmc_params)

# Choose the sampled parameters
selection = choices(mcmc_df.index, weights=mcmc_df["weight"], k=10)
selected_params = []
for i_param_set in selection:
    selected_params.append(get_param_dict_from_run_index(params_df, i_param_set))

# Run one arbitrary selected parameter set to check everything's working
final_params = project.param_set.baseline.update(selected_params[0], calibration_format=True)
model = project.run_baseline_model(final_params)